In [8]:
class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
        self.parent = None
        self.direction = None
        
        self.H = 0
        self.G = 0
        

DXDYDR = [[0, -1, "L"],
          [0,  1, "R"],
          [1,  0, "D"],
          [-1, 0, "U"]]
        
def get_children(x, y):
    children = []
    for dx, dy, dr in DXDYDR:
        if (x+dx, y+dy) in grid:
            child = grid[(x+dx,y+dy)]
            
            children.append([child, dr])  
    return children


def manhattan(point1, point2):
    return abs(point1.x - point2.x) + abs(point1.y - point2.y)

def pythagorean(point1, point2):
    return (point1.x - point2.x)**2 + (point1.y - point2.y)**2


def aStar(start, end):
    #The open and closed sets
    openset = set()
    closedset = set()

    #Current point is the starting point
    current = start
    
    #Add the starting point to the open set
    openset.add(current)

    #While the open set is not empty
    while openset:
        #Find the item in the open set with the lowest G + H score
        
        current = min(openset, key=lambda o:o.G + o.H)

        #If it is the item we want, retrace the path and return it
        if current == end:
            path = ""
            while current.parent:
                path += current.direction
                current = current.parent

                
                                
            
            return path[::-1]

        #Remove the item from the open set
        openset.remove(current)

        #Add it to the closed set
        closedset.add(current)

        #Loop through the node's children/siblings
        for node, direction in get_children(current.x, current.y):
            #If it is already in the closed set, skip it
            if node in closedset:
                continue

            #Otherwise if it is already in the open set
            if node in openset:
                #Check if we beat the G score 
                new_g = current.G + 1
                if node.G > new_g:
                    #If so, update the node to have a new parent
                    node.G = new_g
                    node.parent = current
                    node.direction = direction
            else:
                #If it isn't in the open set, calculate the G and H score for the node
                node.G = current.G + 1
                node.H = pythagorean(node, end)

                #Set the parent to our current item
                node.parent = current
                node.direction = direction
                #Add it to the set
                openset.add(node)

    #Throw an exception if there is no path
    raise ValueError('No Path Found')

In [9]:
DXDY = [[0,-1],[0,1],[1,0],[-1,0]] 
def bfs(x_start, y_start, x_end, y_end):
    DXDY = [[0,-1],[0,1],[1,0],[-1,0]] 
    
    used = [[False] * N for _ in range(N)]
    used[x_start][y_start] = "start"
    q = deque()
    q.append([x_start,y_start])

    count = 0
    while q:
        x, y = q.popleft()
        for dx, dy in DXDY:
            if 0 <= x + dx < N and 0 <= y + dy < N:
                if board[x + dx][y + dy] == "." and not used[x + dx][y + dy]:
                    used[x + dx][y + dy] = [x,y]
                    if [x + dx, y + dy] == [x_end, y_end]:
                        path = [[x + dx, y + dy]]
                        while used[x][y] != "start":
                            path.append([x,y])
                            x, y = used[x][y]
                        
                        return path[::-1]
                    q.append([x + dx, y + dy])


In [10]:
DXDY = [[0,-1],[0,1],[1,0],[-1,0]] 
def bfs_points(start_point, clusters, cells):
    used = [[False] * N for _ in range(N)]
    
    x_start, y_start = start_point
    used[x_start][y_start] = "start"
    
    q = deque()
    q.append([x_start,y_start])

    count = 0
    while q:
        x, y = q.popleft()
        for dx, dy in DXDY:
            nx = x + dx
            ny = y + dy
            if 0 <= nx < N and 0 <= ny < N:
                if (nx, ny) in cells and not used[nx][ny]:
                    used[nx][ny] = (x,y)
                    
                    if (nx, ny) in clusters:
                        path = [[nx, ny]]
                        back_x, back_y = x, y
                        
                        while used[back_x][back_y] != "start":
                            path.append([back_x,back_y])
                            back_x, back_y = used[back_x][back_y]
                        path.append([back_x,back_y])
                        clusters[start_point][(nx, ny)] = path[::-1]
                        clusters[(nx, ny)][start_point] = path
                        
                        #Если нашли все пути
                        if len(clusters[start_point]) == len(clusters) - 1:
                            return 
                    
                    q.append([nx, ny])

                    
def find_beacon(start, clusters, cells, limit):
    if start in clusters:
        return ["", start]
    used = dict()
    used[start] = [None, "s"]
    
    q = deque()
    q.append(start)
    
    counter = 0
    while q:
        x, y = q.popleft()
        for nx, ny, direction in [(x, y-1, "L"), (x, y+1, "R"), (x-1, y, "U"), (x+1,y, "D")]:
            next_cell = (nx, ny)
            counter += 1
            if counter > limit:
                
                return [None, None]
            
            if next_cell in cells and next_cell not in used:
                used[next_cell] = [(x, y), direction]
                q.append(next_cell)
                
                #
                if next_cell in clusters:
                    backtrack = next_cell
                    path = ""
                    while used[backtrack][1] != "s":
    
                        path += used[backtrack][1]
                        backtrack = used[backtrack][0]
                    
                    return [path[::-1], next_cell]
    print(counter)
    raise ValueError('No Path Found')

In [11]:
from PIL import Image 
import PIL 
import time

import numpy as np
import random
from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline                

In [12]:
%%time
with open('sdc_meetup_2021_tests/10', 'r') as f:
    N, MaxTips, Cost = map(int, f.readline().split())
    
    board = []
    IMG = []
    for i in range(N):
        line = f.readline().strip()
        board.append(line)
        line = line.replace("#", "1")
        line = line.replace(".", "0")
        line = list(map(int, list(line)))
        line = [[128, 128, 128] if i == 1 else [0, 0, 0] for i in line]
        IMG.append(line) 
    
    
    
    
    #ORDERS:
    raw_orders = []
    T, D = map(int, f.readline().split())
    for i in range(T):
        k = int(f.readline())
        for j in range(k):
            x1, y1, x2, y2 = map(int, f.readline().split())
            raw_orders.append([(x1 - 1, y1 - 1), (x2 - 1, y2 - 1)])
        


        
        
        
        


IMG = np.array(IMG, dtype=np.uint8)
print(MaxTips, "- чаевые") 
print(Cost, "- цена робота")
print(T, "- количество итераций")
print(D, "- Количество заказов")

#ANCHORS PART:
cells = set()
for x in range(N):
    for y in range(N):
        if board[x][y] == ".":
            cells.add((x,y))  

3600 - чаевые
1048576 - цена робота
77777 - количество итераций
467760 - Количество заказов
CPU times: user 4.3 s, sys: 145 ms, total: 4.45 s
Wall time: 4.51 s


# Кластеры

In [34]:
import copy
image = copy.deepcopy(IMG) 
orders_map = [[0] * N for _ in range(N)]

for start, end in raw_orders:
    x1, y1 = start
    x2, y2 = end
    orders_map[x1][y1] += 1
    
    #image[x1][y1][1] += 1 #зеленый
    
    #image[x2][y2][0] += 1 #красный        


raw_clusters = []
for x in range(N):
    for y in range(N):
        if orders_map[x][y] > 0:
            #if 120 < x < 670 and 300 < y < 700:
                raw_clusters.append((x,y))

random.shuffle(raw_clusters)


for x1,y1 in raw_clusters:
    for x2, y2 in raw_clusters:
        if [x1,y1] == [x2,y2]:
            continue
            
        if abs(x1 - x2) + abs(y1 - y2) <= 5:
            
            if (x2, y2) in raw_clusters:
                raw_clusters.remove((x2,y2))
        
raw_clusters = set(raw_clusters)        
print(len(raw_clusters))



        


for point in raw_clusters:
    x, y = int(point[0]), int(point[1])
    
    for dx, dy in [[0, -1],[0,1],[1,0],[-1,0],[1,1],[1,-1],[-1,1],[-1,-1]]:
        image[x + dx][y + dy] = [0, 0, 255]
        image[x + dx*2][y + dy*2] = [0, 0, 255]


102


In [29]:
# Use PIL to create an image from the new array of pixels
new_image = Image.fromarray(image, "RGB") 
new_image.save('finish_clusters.png')

In [35]:
%%time
clusters = {cluster : {} for cluster in raw_clusters}
for point in clusters.keys():
    bfs_points(point, clusters, cells)
    
    
    
POINTS_D = {(0, -1) : "L",
            (0, 1) : "R",
            (1, 0) : "D",
            (-1,0) : "U"}
for point1 in clusters.keys():
    for point2 in clusters[point1].keys():
        point1_to_point2 = clusters[point1][point2]
        
        path = ""
        for i in range(len(point1_to_point2) - 1):
            x1, y1 = point1_to_point2[i]
            x2, y2 = point1_to_point2[i+1]
            path += POINTS_D[(x2 - x1, y2 - y1)]
        clusters[point1][point2] = path

with open('new_clusters.py', 'w') as f:
    print('clusters = ', clusters, file=f)

CPU times: user 37.8 s, sys: 152 ms, total: 37.9 s
Wall time: 38.3 s


In [17]:
print(board[441][622])


.


# Обработка заказов

In [36]:
%%time
LIMIT = 1000


count = 0
banned = set()
for start_order, end_order in raw_orders[:]:
    if start_order not in banned:
        _, start_cluster = find_beacon(start_order, clusters, cells, limit = LIMIT)
        _, end_cluster = find_beacon(end_order, clusters, cells, limit = LIMIT)
        if start_cluster and end_cluster:
            count += 1
        else:
            banned.add(start_order)
        


print(len(raw_orders))
print(count)



467760
467760
CPU times: user 5.32 s, sys: 0 ns, total: 5.32 s
Wall time: 5.32 s


In [39]:
def show_paths(R, robots):
    paths = [""] * R
    
    for robot in robots:
        path_60  = robot.get_path()
        paths[robot.idx] += path_60

    #for path in paths:
    #    print(path)
        
        

class Robot:
    def __init__(self, idx, cell):
        self.idx = idx #Его индекс
        self.path = "" #Его путь
        self.iter = 0
        self.position = cell


    def get_path(self):
        cur_path = self.path[self.iter:self.iter+60]
        self.iter += 60
        if len(cur_path) < 60:
            
            self.path = ""
            self.iter = 0
            cur_path += "S" * (60 - len(cur_path))
            
            
        
        return cur_path
        


In [40]:
#Заказы и пути к ним
REVERSE_DIR = {"D" : "U",
              "U" : "D",
              "R" : "L",
              "L" : "R"}

def reverse_path(path):
    #path = "DDRRL" -> "UULLR" -> "RLLUU"
    rev_path = ""
    for direction in path[::-1]:
        rev_path += REVERSE_DIR[direction]
    return rev_path



class Order:
    def __init__(self, start, end, iter_start):
        self.start = start
        self.end = end
        self.iter_start = iter_start #Время появления заказа
        
        
        self.beacon_to_start = None
        self.start_to_beacon = None
        self.beacon_to_end = None
        self.end_to_beacon = None
        self.cluster_start = None
        self.cluster_end = None
        
        self.dilevery_time = 0
        
        self.path = ""
        
        
    
    def build_path(self, clusters):
        
        
        #1. Путь от кластера (где есть робот) до заказа start
        self.path += self.beacon_to_start
        self.path += "T" # Забрать заказ
        
        #2. Путь от заказа start до кластера
        self.path += self.start_to_beacon
        
        #3. Путь от кластера start до кластера end
        
        self.path += clusters[self.cluster_start][self.cluster_end]
        
        
        #4. Путь от кластера end до заказа end
        self.path += self.beacon_to_end
        
        self.dilevery_time += len(self.path)
        
        
        self.path += "P"
        
        #5. Путь от заказа end обратно до кластера
        self.path += self.end_to_beacon
        
        return self.path  
    
    
    def calculate_tip(self, MaxTip, cur_iteration):
        t = 0
        t += (cur_iteration - self.iter_start) * 60
        t += self.dilevery_time
        
        return max(0, MaxTip - t)

In [41]:
print(len(clusters))

102


In [1]:
%%time
#%%capture
# Получение заказов

with open('sdc_meetup_2021_tests/10', 'r') as f:
    EARNINGS = 0
    DELIVERED = 0
    
    N, MaxTips, Cost = map(int, f.readline().split())
    
    board = []
    for i in range(N):
        line = f.readline().strip()
        board.append(line)
    
    T, D = map(int, f.readline().split())
    
    
    #ROBOTS#############################
    #Выбираем 100 случайных кластеров
    R = 100 # Количество роботов
    LIMIT = 10000
    
    
    #EARNINGS -= Cost * R
    #print(R, flush = True)

    random_clusters = random.sample(clusters.keys(), R//2) + random.sample(clusters.keys(), R//2)
    

    
    
    robots = set()
    
    
    #Заполняем ячейки списками свободных роботов
    for idx, cell in enumerate(random_clusters):
        robots.add(Robot(idx, cell))

        #print(cell[0]+1, cell[1]+1, flush = True)
        

    
    #ANCHORS PART:
    cells = set()
    for x in range(N):
        for y in range(N):
            if board[x][y] == ".":
                cells.add((x,y))  

    
    #ORDERS:
    orders = {cell : deque() for cell in clusters.keys()}
  
   
    for i in range(T):
        
        
        k = int(f.readline())
        for j in range(k):
            x1, y1, x2, y2 = map(int, f.readline().split())
            order_start = (x1 - 1, y1 - 1)
            order_end = (x2 - 1, y2 - 1)
   
            
            path_start, cluster_start = find_beacon(order_start, clusters, cells, limit = LIMIT)
   
            
            
            path_end, cluster_end = find_beacon(order_end, clusters, cells, limit = LIMIT)
            
            
  
            order = Order(order_start, order_end, i + 1)
    
            
            
            order.start_to_beacon = path_start
            order.beacon_to_start = reverse_path(path_start)
            order.end_to_beacon = path_end
            order.beacon_to_end = reverse_path(path_end)
            order.cluster_start = cluster_start
            order.cluster_end = cluster_end
            
            orders[cluster_start].append(order)
        
        #Для каждого робота проверить, есть ли заказ
        for robot in robots:
            if robot.path == "":
                START_POSITION = robot.position
                if len(orders[robot.position]) > 0:

                    order = orders[robot.position].popleft()

                    robot.path = order.build_path(clusters)
                    robot.position = order.cluster_end
                    
                    
                    #INFO
                    DELIVERED += 1
                    EARNINGS += order.calculate_tip(MaxTips, i + 1)
                    
                    #CHECKER(board, START_POSITION, robot.position, robot.path, order.start, order.end)

                else:
                    #Посылаем в рандомный cell

                    random_cluster = random.sample(clusters.keys(), 1)[0]
                    while random_cluster == robot.position:
                        random_cluster = random.sample(clusters.keys(), 1)[0]

                    robot.path = clusters[robot.position][random_cluster]
                    robot.position = random_cluster
                
                    #       Поле   Начало          Конец
                    #CHECKER(board, START_POSITION, robot.position, robot.path)
   
               
 
            
        show_paths(R, robots)
        
        
print(EARNINGS)
print(DELIVERED)    



NameError: name 'random' is not defined

x = 532 - 1
y = 553 - 1
#LDDDDLLLLLLULLULLULLULLULLULLLUULLLLLLLLLLLLLLLLLLULLLULLULU

def replace_str_index(text, index, replacement):
    return '%s%s%s'%(text[:index],replacement,text[index+1:])

board[x] = replace_str_index(board[x], y, "$")

for i in range(515, 540):
    print(board[i][520:560])

    

In [231]:
p = ""
CHECKER(board, (374, 509), (0,0), p)

[348, 511]
DRRRRRRRUURUURUUUURRUURUURUUUUULULLULLUUUULULLUULLLULULLULLU


ValueError: WALL

In [259]:
def draw_path(points, PROBLEM):
    image = copy.deepcopy(IMG)
    for x, y in points:
        image[x][y][0] = 100
    
    x, y = PROBLEM
    image[x][y] = [0, 0, 255]
    
    new_image = Image.fromarray(image, "RGB") 
    new_image.save('ERROR.png')

ACTION_D = {"U" : (-1, 0),
            "D" : (1, 0),
            "R" : (0, 1),
            "L" : (0, -1)}

def CHECKER(board, start_position, end_position, path, order_start = None, order_end = None):
    result = None
    
    x, y = start_position
    POINTS = [[x,y]]
    PROBLEM = order_start
    for action in path:
        if action == "U":
            x -= 1
        elif action == "D":
            x += 1
        elif action == "R":
            y += 1
        elif action == "L":
            y -= 1
        POINTS.append([x,y])

        if board[x][y] == "#":
            PROBLEM = [x,y]
            result = 'WALL'
        
        elif action == "T" and order_start != (x, y):
            PROBLEM = [x,y]
            result = "NO PACKAGE"
            
        elif action == "P" and order_end != (x, y):
            PROBLEM = [x,y]
            result = "NO PLACE TO DELIVER"
    
    if (x, y) != end_position:
        PROBLEM = [x,y]
        result = "DIDN'T REACH DESTINATION"
            
    if result is not None:
    #if PROBLEM:
        draw_path(POINTS, PROBLEM)
        print(PROBLEM)
        print(path)
        raise ValueError(result)

## Путь

*Кластер -> заказ_старт -> кластер -> заказ_финиш -> кластер*

# Аналитика

# KMeans

In [12]:
from sklearn.cluster import KMeans
#K-means для всех точек получения заказа:
orders_start = np.array([i[0] for i in orders])

kmeans = KMeans(n_clusters = k).fit(orders_start)


TypeError: 'Order' object is not subscriptable

In [ ]:
#Нарисуем точки старта заказов и центры кластеров

for point in orders_start:
    x, y = int(point[0]), int(point[1])
    
    for dx, dy in [[0, -1],[0,1],[1,0],[-1,0],[1,1],[1,-1],[-1,1],[-1,-1]]:
        image[x + dx][y + dy][0] += 1 #фиолетовый
        


for point in kmeans.cluster_centers_:
    x, y = int(point[0]), int(point[1])
    
    for dx, dy in [[0, -1],[0,1],[1,0],[-1,0],[1,1],[1,-1],[-1,1],[-1,-1]]:
        image[x + dx][y + dy] = [50, 255, 255]
        image[x + dx*2][y + dy*2] = [50, 255, 255]

In [ ]:
from sklearn.cluster import KMeans
#K-means для всех точек получения заказа:
orders_end = np.array([i[1] for i in orders])

kmeans = KMeans(n_clusters = k).fit(orders_end)

In [ ]:
#Нарисуем точки старта заказов и центры кластеров
for point in orders_end:
    x, y = int(point[0]), int(point[1])
    
    for dx, dy in [[0, -1],[0,1],[1,0],[-1,0],[1,1],[1,-1],[-1,1],[-1,-1]]:
        image[x + dx][y + dy][0] += 1 #фиолетовый
        


for point in kmeans.cluster_centers_:
    x, y = int(point[0]), int(point[1])
    
    for dx, dy in [[0, -1],[0,1],[1,0],[-1,0],[1,1],[1,-1],[-1,1],[-1,-1]]:
        image[x + dx][y + dy] = [50, 255, 255]
        image[x + dx*2][y + dy*2] = [50, 255, 255]

In [ ]:
def draw_points(points):
    for point in points.values():
        for path in point.values():
            for x, y in path:
                image[x][y][0] += 100
    
    for point in points.keys():
        x, y = point
        for dx, dy in [[0, -1],[0,1],[1,0],[-1,0],[1,1],[1,-1],[-1,1],[-1,-1]]:
            image[x+dx][y+dy] = [50, 255, 255]
            image[x+dx*2][y+dy*2] = [50, 255, 255]

draw_points(clusters)

In [18]:
"""
dist = 0
for i in range(100):
    if i % 10 == 0:
        print(i)
        print(dist/(i+1))
    start, end = orders[i]


    path = bfs(*start, *end)
    dist += len(path)

    for x, y in path:
        image[x][y][0] += 60
    
    
    #for dx, dy in DXDY:
        
    image[start[0]][start[1]] = [0, 0, 255]
    image[end[0]][end[1]] =     [0, 255, 0]
   
"""


#Convert all obtainable the points to instances of Node
grid = {}
for x in range(N):
    for y in range(N):
        if board[x][y] == ".":
            grid[(x,y)] = Node(x,y)

print(len(grid))

for i in range(50):
    start, end = orders[i]
    path = aStar(grid[start], grid[end])
    for node in grid.values():
        node.parent = None
        node.direction = None
    

84324


KeyError: 0

05. Среднее растояние от пункта выдачи, до пункта доставки заказа ~682
08. 531

In [ ]:
70000 * 3000 * 5